In [3]:
import nibabel as nib
import numpy as np

In [4]:
seed_aseg = nib.load(snakemake.input.seed_aseg)
label_num = snakemake.params.seed_label

#extract seed aseg data
seed_vol = seed_aseg.get_fdata()

#set to 1 where label_num, 0 otherwise
seed_vol = np.where(seed_vol == label_num, 1, 0)

#put in a new nibabel img
seed_single = nib.Nifti1Image(seed_vol,seed_aseg.affine,seed_aseg.header)

Use nilearn's built-in visualization to check the seed image, using the T1w as background

In [6]:
from nilearn.plotting import view_img

view = view_img(seed_single, snakemake.input.t1w,threshold=1)
view


Now we can use that image to make a masker in Nilearn.
Maskers are used extract data from fmri timeseries. The NiftiLabelsMasker uses the segmentation to get a timeseries for that binary mask.

In [40]:
from nilearn.input_data import NiftiLabelsMasker

#create the masker that will sample the timeseries
seed_masker = NiftiLabelsMasker(labels_img=seed_single, mask_img=snakemake.input.brain_mask, standardize='zscore', resampling_target='data')

#sample the timeseries by providing the denoised input (note, could have alternatively provided confounds here)
seed_timeseries = seed_masker.fit_transform(snakemake.input.denoised)



We can use the NiftiMasker to get timeseries for each voxel in the mask (brain mask in this case):

In [41]:
from nilearn.input_data import NiftiMasker

#similarly, get timeseries for all voxels in the brain:
brain_masker = NiftiMasker( mask_img=snakemake.input.brain_mask, standardize='zscore', smoothing_fwhm=snakemake.params.fwhm)
brain_timeseries = brain_masker.fit_transform(snakemake.input.denoised)



In [42]:
print(brain_timeseries.shape)
print(seed_timeseries.shape)

In [43]:
import matplotlib.pyplot as plt

plt.plot(seed_timeseries)


In [45]:
#seed to voxel correlation:
corr = np.dot(seed_timeseries.T,brain_timeseries) / seed_timeseries.shape[0]
corr.shape

#bring back to image space
corr_img = brain_masker.inverse_transform(corr)
corr_img.shape
view = view_img(corr_img,snakemake.input.t1w, threshold=snakemake.params.r_threshold)
view

In [46]:
from nilearn.image import threshold_img

#threshold
corr_img_th = threshold_img(corr_img,snakemake.params.r_threshold)

#save the connectivity to nifti file
corr_img_th.to_filename(snakemake.output.seed_fconn)